In [1]:
from pyspark.sql import SparkSession

# Initialize SparkSession with Hive support and S3A configs
"""
All required configs are stored in /configs/spark/spark-defaults.conf.
To Override, use .config() ex: .config("spark.hadoop.fs.s3a.access.key", "test")

"""
spark = SparkSession.builder \
    .appName("Spark S3 Parquet Hive Table Example") \
    .enableHiveSupport() \
    .getOrCreate()

print("Hadoop version:", spark.sparkContext._jvm.org.apache.hadoop.util.VersionInfo.getVersion())


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/01 06:54:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Hadoop version: 3.4.1


In [2]:

# spark.sql("DESCRIBE FORMATTED default.northwind_orders").show(truncate=False)

# Path to your Parquet data in S3
parquet_path = "s3a://demo-bucket/northwind/order_details.parquet"  # Change to your actual folder

# Read the Parquet file(s)
df = spark.read.parquet(parquet_path)

# Show the DataFrame (optional, for verification)
df.show()



25/07/01 06:54:54 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


+-------+---------+---------+--------+--------+
|OrderID|ProductID|UnitPrice|Quantity|Discount|
+-------+---------+---------+--------+--------+
|  10248|       11|     14.0|      12|     0.0|
|  10248|       42|      9.8|      10|     0.0|
|  10248|       72|     34.8|       5|     0.0|
|  10249|       14|     18.6|       9|     0.0|
|  10249|       51|     42.4|      40|     0.0|
|  10250|       41|      7.7|      10|     0.0|
|  10250|       51|     42.4|      35|    0.15|
|  10250|       65|     16.8|      15|    0.15|
|  10251|       22|     16.8|       6|    0.05|
|  10251|       57|     15.6|      15|    0.05|
|  10251|       65|     16.8|      20|     0.0|
|  10252|       20|     64.8|      40|    0.05|
|  10252|       33|      2.0|      25|    0.05|
|  10252|       60|     27.2|      40|     0.0|
|  10253|       31|     10.0|      20|     0.0|
|  10253|       39|     14.4|      42|     0.0|
|  10253|       49|     16.0|      40|     0.0|
|  10254|       24|      3.6|      15|  

In [3]:

# Write as a Hive table (metadata goes to Hive Metastore, data stays in S3)
df.write.mode("overwrite").saveAsTable("default.northwind_orders1")

# Query the table to verify
result = spark.sql("SELECT * FROM default.northwind_orders1")
result.show()



25/07/01 06:55:20 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.


+-------+---------+---------+--------+--------+
|OrderID|ProductID|UnitPrice|Quantity|Discount|
+-------+---------+---------+--------+--------+
|  10248|       11|     14.0|      12|     0.0|
|  10248|       42|      9.8|      10|     0.0|
|  10248|       72|     34.8|       5|     0.0|
|  10249|       14|     18.6|       9|     0.0|
|  10249|       51|     42.4|      40|     0.0|
|  10250|       41|      7.7|      10|     0.0|
|  10250|       51|     42.4|      35|    0.15|
|  10250|       65|     16.8|      15|    0.15|
|  10251|       22|     16.8|       6|    0.05|
|  10251|       57|     15.6|      15|    0.05|
|  10251|       65|     16.8|      20|     0.0|
|  10252|       20|     64.8|      40|    0.05|
|  10252|       33|      2.0|      25|    0.05|
|  10252|       60|     27.2|      40|     0.0|
|  10253|       31|     10.0|      20|     0.0|
|  10253|       39|     14.4|      42|     0.0|
|  10253|       49|     16.0|      40|     0.0|
|  10254|       24|      3.6|      15|  

In [4]:
# Stop the Spark session
spark.stop()